In [2]:
import json
import chromadb
from function import get_market_info_token, get_portfolio

In [1]:
from openai import OpenAI

model_repo = {
    'llama3': {
        'client': OpenAI(
            api_key = 'YOUR_API_KEY',
            base_url = 'YOUR_URL'
        ),
        'model': 'Llama3-70b-8192'
    }
}

In [5]:
tools = [
    {
        "function": {
            "name": "get_market_info_token",
            "description": "Retrieve market information for a specific token, including Price, Market cap, FDV, rank, category, chain, using the CoinGecko API.\n"
            "    \n"
            "    Args:\n"
            f"        tokenSymbol (str): The input token symbol. for example: btc, eth, bnb... MUST be in {symbol_to_id} if existed.\n"
            f"        actionType (str): Type of action from input (market cap, volume, price, fdv).\n"
            f"        interval (str, optional): The interval time is not used in this function as CoinGecko does not provide historical data directly through the simple price endpoint.\n"
            "    \n"
            "    Returns:\n"
            f"        The market information of the specified token.\n",
            "parameters": {
                "properties": {
                    "tokenSymbol": {"type": "string"},
                    "actionType": {"type": "string"},
                    "interval": {"type": "string"},
                },
                "required": ["tokenSymbol", "actionType"],
                "type": "object",
            },
        },
        "type": "function",
    },
    {
        "function": {
            "name": "get_portfolio",
            "description": "Get the current portfolio or balance of the user. This function must be called before any actions like swapping to get the exact user balance.\n"
            "    \n"
            "    Args:\n"
            f"        threadId (str): The thread id of the user.\n"
            "    \n"
            "    Returns:\n"
            f"        The portfolio/balance of the user.\n",
            "parameters": {
                "properties": {
                    "threadId": {"type": "string"},
                },
                "required": ["threadId"],
                "type": "object",
            },
        },
        "type": "function",
    }
]

In [6]:
def inference(messages, **kwargs):
    model = kwargs.get('model', "llama3")
    max_tokens = kwargs.get('max_tokens', 1500)
    temperature = kwargs.get('temperature', 0.0)
    stream = kwargs.get('stream', False)

    chat_response = model_repo[model]['client'].chat.completions.create(
        model=model_repo[model]['model'],
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        tools=tools,
        tool_choice="auto",
        stream=stream
    )

    response_text = chat_response.choices[0].message
    tool_calls = response_text.tool_calls
    messages.append(response_text)
    if tool_calls:
        available_functions = {
            "get_market_info_token": get_market_info_token,
            "get_portfolio": get_portfolio
        }
        messages.append(response_text)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = model_repo[model]['client'].chat.completions.create(
            model=model_repo[model]['model'],
            messages=messages,
            stream=stream,
        )
        if stream:
            for chunk in second_response:
                text = chunk.choices[0].delta.content
                if text:
                    yield text
        else:
            response_text = second_response.choices[0].message.content
            yield response_text
    else:
        yield response_text.content

In [7]:
db_examples = [
    "1st customer mean userId u001",
    "35th customer mean userId u035",
    "first customer mean userId u001",
    "second customer mean userId u002",
]

In [8]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="db_examples")

db_id = []
for i in range(len(db_examples)):
    db_id.append("id" + str(i))

collection.add(documents=db_examples, ids=db_id)

In [12]:
content_prompt = "Can you give me the price of Bitcoin?"

examples = collection.query(
    query_texts= content_prompt,  # Chroma will embed this for you
    n_results=3,  # how many results to return
)
examples = examples.get("documents")

In [13]:
system_prompt = """
You are a helpful assistant.
If you can't call the function, you should tell the user to change the prompt
You are provided with function signatures within <tools></tools> XML tags.
Use them if required

Here are the available tools:
<tools> {tools} </tools>

Here are some examples:
{examples}

Use the following pydantic model json schema for each tool call you will make: {'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name'], 'title': 'FunctionCall', 'type': 'object'}

Make sure that the json object above with code markdown block is parseable with json.loads() and the XML block with XML ElementTree.
""".replace(
    "{tools}", str(tools)
). replace("{examples}", str(examples))

In [14]:
for chunk in inference(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": content_prompt}
    ], temperature=0, strem=True):
    print(chunk, end='')

The current price of Bitcoin is $66,290.